In [2]:
import scgen
print(scgen.__version__)
# Import package 
# Main using package here is scanpy 
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
# sc.settings.set_figure_params(dpi=300, frameon=False)  # low dpi (dots per inch) yields small inline figures

scanpy==1.4 anndata==0.6.18 numpy==1.16.2 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
# Create folder to save the results 
import os
dirname = os.getcwd()
print(dirname)
data_dir = os.path.join(dirname, './data/dataset2_cellatlas/')
if not os.path.exists('./results_cellatlas/'): os.makedirs('./results_cellatlas/')

if not os.path.exists('./results_cellatlas/scGen_results/'): os.makedirs('./results_cellatlas/scGen_results/')
    
save_dir = os.path.join(dirname, './results_cellatlas/scGen_results/')
def save_images(basename):
    if not os.path.exists('./figures_cellatlas/'): os.makedirs('./figures_cellatlas/')
    
    if not os.path.exists('./figures_cellatlas/scGen_results/'): os.makedirs('./figures_cellatlas/scGen_results/')
    outname = './figures_cellatlas/scGen_results/' + basename + '.png'
    pl.savefig(outname, dpi=150)
    pl.close()

/data_n3/home/hoatran/python-workspace/dca_scGen


In [4]:
# corrected_adata = sc.read_h5ad(os.path.join(data_dir,'correction_dataset1_uc3.h5ad'))
# sc.tl.pca(corrected_adata, svd_solver='arpack', n_comps=20)
# print(corrected_adata)
# corrected_adata.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
# save_dir = os.path.join(dirname, './results/scGen_results/')
# corrected_adata.write_csvs(save_dir, skip_data=True)

In [5]:
# read data from read count text table, data from R: genes x cells
adata = sc.read_text(os.path.join(data_dir,'dataset1_sm_uc3.txt'),delimiter='\t',first_column_names=True,dtype='float64')
print(adata)
print(len(adata.obs_names))
print(adata.var_names)
# Read sample into a pandas series
sample_adata = pd.read_csv(os.path.join(data_dir,'sample_sm_uc3.txt'),header=0, index_col=0, sep='\t')
print(sample_adata.values.shape)
print(sample_adata.keys())

In [6]:
# Data have format genes x cells
# But input for scanpy package require the format of cells x genes, samples x features
# We need to copyt data into new adata object with inverse raw data 
# adata1 = sc.AnnData(adata.X.T)
# adata1.var_names = adata.obs_names
# adata1.obs_names = adata.var_names
# adata1
# # Save label information into adata object, same idea as Seurat 
# adata1.obs['cell_type'] = sample_adata.loc[adata1.obs_names,['celltype']]
# adata1.obs['batch'] = sample_adata.loc[adata1.obs_names,['batch']]
# print(len(adata1.obs['cell_type']))
# print(len(adata1.obs['batch']))
# Save output into h5ad, easy to access 
# adata1.write_h5ad(os.path.join(data_dir,'dataset1_uc3.h5ad'))

In [7]:
adata1 = sc.read_h5ad(os.path.join(data_dir,'dataset1_uc3.h5ad'))
print('Read and filter data')
# adata = sc.read_h5ad(os.path.join(data_dir,'filtered_dataset1_uc3.h5ad'))
# adata1

Read and filter data


In [8]:
import time
from datetime import timedelta

# Filtering data 
sc.pp.filter_cells(adata1, min_genes=300)
sc.pp.filter_genes(adata1, min_cells=5)
adata1

filtered out 11 cells that have less than 300 genes expressed
filtered out 9999 genes that are detected in less than 5 cells


AnnData object with n_obs × n_vars = 565 × 16594 
    obs: 'cell_type', 'batch', 'n_genes'
    var: 'n_cells'

In [9]:
# adata1.write_h5ad(os.path.join(data_dir,'filtered_dataset1_uc3.h5ad'))
# Visualization of raw data
# sc.tl.pca(adata1, svd_solver='arpack')
# sc.pp.neighbors(adata1,n_neighbors=10, n_pcs=20)
# sc.tl.umap(adata1)
# sc.pl.umap(adata1, color=["cell_type","batch"], wspace=.3, show=False)
# save_images('umap_filtered_dataset1_uc3')

In [10]:
# Transform and scale data
sc.pp.log1p(adata1)
sc.pp.scale(adata1)
# sc.pp.neighbors(adata1,n_neighbors=10, n_pcs=20)
# color_group = ["batch","cell_type"]
# plotTSNE(adata1, color_group, 20, 110, 'tsne_filtered_dataset1_uc3')

In [11]:
# filter_result = sc.pp.filter_genes_dispersion(adata1.X, min_mean=0.0125, max_mean=3, min_disp=0.5)
# sc.pl.filter_genes_dispersion(filter_result, show=False)
# save_images('scgen_filter_genes_dispersion')
# adata1 = adata1[:, filter_result.gene_subset]
# # color_group = ["cell_type","batch"]
# plotUMAP(adata1, color_group, 'umap_filtered_dataset1_uc3')

In [16]:
t1 = time.time()
# Initialize scGen with input is number of genes
import scgen
network = scgen.VAEArith(x_dimension=adata1.shape[1], model_path="./results/batch")
# Need to check batch_size

In [17]:
# Train scGen with nb epochs = 100
# Requirement: adata should contain 2 vector: adata.obs["cell_type"] and adata.obs["batch"]
network.train(train_data=adata1, n_epochs=100, batch_size=30)

In [18]:
from scipy import sparse
import anndata
# Using trained model to correct, normalize data 
# Using batch removal function from scGen package 

#corrected_adata = scgen.batch_removal(network, total_ann)

# In case this function does not work, replace batch_removal function by this function: batch_removal_v2
# Hoa Tran
def batch_removal_v2(network, adata):
    if sparse.issparse(adata.X):
        latent_all = network.to_latent(adata.X.A)
    else:
        latent_all = network.to_latent(adata.X)
    adata_latent = anndata.AnnData(latent_all)
    adata_latent.obs["cell_type"] = adata.obs["cell_type"].tolist()
    adata_latent.obs["batch"] = adata.obs["batch"].tolist()
    adata_latent.obs["cell_name"] = adata.obs["cell_name"].tolist()   #Hoa keep cell name infos
    unique_cell_types = np.unique(adata_latent.obs["cell_type"])
    shared_ct = []
    not_shared_ct = []
    for cell_type in unique_cell_types:
        temp_cell = adata_latent[adata_latent.obs["cell_type"] == cell_type]
        if len(np.unique(temp_cell.obs["batch"])) < 2:
            cell_type_ann = adata_latent[adata_latent.obs["cell_type"] == cell_type]
            not_shared_ct.append(cell_type_ann)
            continue
        temp_cell = adata_latent[adata_latent.obs["cell_type"] == cell_type]
        batch_list = {}
        batch_ind = {}
        max_batch = 0
        max_batch_ind = ""
        batches = np.unique(temp_cell.obs["batch"])
        for i in batches:
            temp = temp_cell[temp_cell.obs["batch"] == i]
            temp_ind = temp_cell.obs["batch"] == i
            if max_batch < len(temp):
                max_batch = len(temp)
                max_batch_ind = i
            batch_list[i] = temp
            batch_ind[i] = temp_ind
        max_batch_ann = batch_list[max_batch_ind]
        for study in batch_list:
            delta = np.average(max_batch_ann.X, axis=0) - np.average(batch_list[study].X, axis=0)
            batch_list[study].X = delta + batch_list[study].X
            temp_cell[batch_ind[study]].X = batch_list[study].X
        shared_ct.append(temp_cell)
    all_shared_ann = anndata.AnnData.concatenate(*shared_ct, batch_key="concat_batch")
    del all_shared_ann.obs["concat_batch"]
    if len(not_shared_ct) < 1:
        corrected = anndata.AnnData(network.reconstruct(all_shared_ann.X, use_data=True))
        corrected.obs["cell_type"] = all_shared_ann.obs["cell_type"].tolist()
        corrected.obs["batch"] = all_shared_ann.obs["batch"].tolist()
        corrected.obs["cell_name"] = all_shared_ann.obs["cell_name"].tolist() #Hoa keep cell name infos
        corrected.var_names = adata.var_names.tolist()
        corrected.obs_names = corrected.obs['cell_name'] #Hoa assign cell name infos
        return corrected
    else:
        all_not_shared_ann = anndata.AnnData.concatenate(*not_shared_ct, batch_key="concat_batch")
        all_corrected_data = anndata.AnnData.concatenate(all_shared_ann, all_not_shared_ann, batch_key="concat_batch")
        del all_corrected_data.obs["concat_batch"]
        corrected = anndata.AnnData(network.reconstruct(all_corrected_data.X, use_data=True), )
        corrected.obs["cell_type"] = all_shared_ann.obs["cell_type"].tolist() + all_not_shared_ann.obs[
            "cell_type"].tolist()
        corrected.obs["batch"] = all_shared_ann.obs["batch"].tolist() + all_not_shared_ann.obs["batch"].tolist()
        corrected.obs["cell_name"] = all_shared_ann.obs["cell_name"].tolist() + all_not_shared_ann.obs[
            "cell_name"].tolist()     #Hoa keep cell name infos
        corrected.var_names = adata.var_names.tolist()
        corrected.obs_names = corrected.obs['cell_name'] #Hoa assign cell name infos
        return corrected



In [19]:
# Correct data using batch_removal function
# Input: adata and network model 
adata1.obs['cell_name'] = adata1.obs_names
corrected_adata = batch_removal_v2(network, adata1)
t2 = time.time()
print('Took '+str(timedelta(seconds=t2-t1)))
# corrected_adata = scgen.batch_removal(network, adata1)
# For verification
# print(corrected_adata.obs['cell_name'][350:400])
# print(corrected_adata.obs['cell_type'][350:400])
corrected_adata.obs_names[1:10]

Took 0:02:13.048766


Index(['DoubleNeg_P10_S74', 'DoubleNeg_P10_S75', 'DoubleNeg_P10_S76',
       'DoubleNeg_P10_S77', 'DoubleNeg_P10_S78', 'DoubleNeg_P10_S79',
       'DoubleNeg_P10_S80', 'DoubleNeg_P10_S81', 'DoubleNeg_P10_S82'],
      dtype='object', name='cell_name')

In [43]:
print(t1)
print(t2)
time_taken = t2 - t1
time_taken_mins = divmod(time_taken, 60)
time_taken_hours, rest = divmod( time_taken, 3600)
hours_mins, hours_secs = divmod( rest, 60)
print('Took seconds: '+str(timedelta(seconds=round(time_taken))))
print('Took minutes: '+str(time_taken_mins))
print('Took hours_minutes_seconds: ',str(time_taken_hours),str(hours_mins),str(hours_secs))
usecase_name = 'scGen_uc3'
filename = 'scGen_uc3_extime.csv'

data = {'use_case':usecase_name, 'exetime_secs':str(round(time_taken)),
       'exetimehours': str(time_taken_hours),
       'exetimemins': str(hours_mins),
       'exetimesecs':str(round(hours_secs))} 
  
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['extime'])
print(df)
save_dir = '/data_n3/home/hoatran/python-workspace/dca_scGen/results/scGen_results/'
df.to_csv(save_dir+filename) 

1557282884.3292305
1557283017.3779964
Took seconds: 0:02:13
Took minutes: (2.0, 13.048765897750854)
Took hours_minutes_seconds:  0.0 2.0 13.048765897750854
         use_case exetime_secs exetimehours exetimemins exetimesecs
extime  scGen_uc3          133          0.0         2.0          13


In [40]:
save_dir

'/data_n3/home/hoatran/python-workspace/dca_scGen/./results/'

In [44]:
sc.tl.pca(corrected_adata, svd_solver='arpack', n_comps=20)
corrected_adata.obsm['X_pca'] *= -1 # multiply by -1 to match Seurat, same scale as total_ann

In [59]:
# Corrected data have same dimensions as input data
print(corrected_adata)
corrected_adata.write_h5ad(os.path.join(save_dir,'scgen_correction_dataset1_uc3.h5ad'))

AnnData object with n_obs × n_vars = 565 × 16594 
    obs: 'cell_type', 'batch', 'cell_name'
    uns: 'pca', 'neighbors', 'cell_type_colors', 'batch_colors'
    obsm: 'X_pca', 'X_umap', 'X_tsne'
    varm: 'PCs'


In [46]:
save_dir = os.path.join(dirname, 'results/scGen_results/')
corrected_adata.write_csvs(save_dir, skip_data=False)

writing '.csv' files to /data_n3/home/hoatran/python-workspace/dca_scGen/results/scGen_results


In [63]:
corrected_adata.obsm['X_pca'].shape[1]

50

In [65]:
def write_to_csv(mat, genesname, cellsname, filename, save_dir):
    if isinstance(mat, np.ndarray):
        df = pd.DataFrame(mat, columns=genesname, index=cellsname)
    else:
        df = pd.DataFrame(mat.toarray(), columns=genesname, index=cellsname)        
    
    df.to_csv(save_dir+filename)     
filename = 'scGen_corrected_pca.csv'
coln_pca = []
for i in range(corrected_adata.obsm['X_pca'].shape[1]):
    coln_pca.append("X_pca"+str(i+1))
    
# save_dir = os.path.join(dirname, './data/dataset1_uc3/')
write_to_csv(corrected_adata.obsm['X_pca'], coln_pca, corrected_adata.obs_names,filename, save_dir)

In [53]:
sc.tl.pca(adata1, svd_solver='arpack')
adata1.obsm.X_pca *= -1 # multiply by -1 to match Seurat, same scale as corrected_adata
# Compute the silhouette coefficient score, compare the batch mixing level of filtered data and normalized data
from sklearn.metrics import silhouette_score
import seaborn as sns
sil_orig = silhouette_score(adata1.obsm.X_pca[:,:20], adata1.obs['batch'])
# sil_true = silhouette_score(corrected_adata.obsm.X_pca[:, :2], adata_true.obs.Group)
sil_corr = silhouette_score(corrected_adata.obsm.X_pca[:,:20], corrected_adata.obs['batch'])
print(sil_corr)
print(sil_orig)
usecase_name = 'scGen_uc3_silscore'
data_sil = {'use_case':usecase_name, 'sil_filtered':sil_orig,
       'sil_corrected': sil_corr} 
  
# Creates pandas DataFrame. 
df_score = pd.DataFrame(data_sil, index =['silscore'])
print(df_score)
save_dir = '/data_n3/home/hoatran/python-workspace/dca_scGen/results/scGen_results/'
df_score.to_csv(save_dir+usecase_name+'.csv') 
# sil_score_ls = []
# sil_score_ls.append(sil_orig)
# sil_score_ls.append(sil_corr)
# label_sil = ['Filtered', 'scGen_Normalized']
# sns.barplot(x=label_sil, y=sil_score_ls)
# pl.title('Silhouette Coef Batch Mixing')
# save_images('scGen_silhouette_coeff_batch_label') 

0.08013981
0.118949324
                    use_case  sil_filtered  sil_corrected
silscore  scGen_uc3_silscore      0.118949        0.08014


In [48]:
print(corrected_adata)
colnu = []
for i in range(corrected_adata.obsm['X_umap'].shape[1]):
    colnu.append("UMAP"+str(i+1))
df = pd.DataFrame(corrected_adata.obsm['X_umap'], columns=colnu, index=corrected_adata.obs_names)
df['batch'] = pd.Series(corrected_adata.obs['batch'], index=corrected_adata.obs_names)
df['celltype'] = pd.Series(corrected_adata.obs['cell_type'], index=corrected_adata.obs_names)
df.to_csv(sdir_name+'scGen_umap.csv')  

colnt = []
for i in range(corrected_adata.obsm['X_tsne'].shape[1]):
    colnt.append("tSNE_"+str(i+1))

df = pd.DataFrame(corrected_adata.obsm['X_tsne'], columns=colnt, index=corrected_adata.obs_names)
df['batch'] = pd.Series(corrected_adata.obs['batch'], index=corrected_adata.obs_names)
df['celltype'] = pd.Series(corrected_adata.obs['cell_type'], index=corrected_adata.obs_names)
df.to_csv(sdir_name+'scGen_tsne.csv') 

'/data_n3/home/hoatran/python-workspace/dca_scGen/results/scGen_results/'

In [55]:
# print(corrected_adata.obs['cell_type'][1:10])
# print(adata1.obs['cell_type'][1:10])
import pandas as pd

# def write_to_csv(mat, cellsname, genesname, filename, save_dir):
#     if isinstance(mat, np.ndarray):
#         df = pd.DataFrame(mat, columns=cellsname, index=genesname)
#     else:
#         df = pd.DataFrame(mat.toarray(), columns=cellsname, index=genesname)        
    
#     df.to_csv(save_dir+filename) 
    
def write_to_csv(mat, genesname, cellsname, filename, save_dir):
    if isinstance(mat, np.ndarray):
        df = pd.DataFrame(mat, columns=genesname, index=cellsname)
    else:
        df = pd.DataFrame(mat.toarray(), columns=genesname, index=cellsname)        
    
    df.to_csv(save_dir+filename)     
filename = 'scGen_corrected_d1_uc3.csv'
# save_dir = os.path.join(dirname, './data/dataset1_uc3/')
write_to_csv(corrected_adata.X, corrected_adata.var_names, corrected_adata.obs_names,filename, save_dir)

In [56]:
# Check the changement of values before scGen and after batch correction using scGen 
import scipy.stats as st
def describe_data(adata, ax, indx):    
    desc = st.describe(adata.X[indx], axis=ax)
    print('# of observations:', desc.nobs)
    print('min: %d\nmax: %d' % desc.minmax)
    print('mean: %.1f' % desc.mean)
    
# Get one cell from batch 1, check the values before normalization and after   
print("Cell id: 10 in ",adata1.obs['batch'][10])
print("Gene expression values before scGen:")
describe_data(adata1, 0, 10) #cell id 10 in batch 1
print("Gene expression values after scGen:")
describe_data(corrected_adata, 0, 10) #cell id 10 in batch 1

Cell id: 10 in  Batch1
Gene expression values before scGen:
# of observations: 16594
min: -4
max: 18
mean: 0.1
Gene expression values after scGen:
# of observations: 16594
min: 0
max: 9
mean: 0.1


In [57]:
# Get one cell from batch 2, check the values before normalization and after
print("Cell id: 500 in ",adata1.obs['batch'][500])
print("Gene expression values before scGen:")
describe_data(adata1, 0, 500)      # cell id 500 in batch 2  
print("Gene expression values after scGen:")
describe_data(corrected_adata, 0, 500)      # cell id 500 in batch 2  

Cell id: 500 in  Batch2
Gene expression values before scGen:
# of observations: 16594
min: -7
max: 17
mean: -0.1
Gene expression values after scGen:
# of observations: 16594
min: 0
max: 6
mean: 0.1


In [58]:
# Function to plot TSNE
def plotTSNE(adata, color_group, n_pcs=20, perplexity=90, save_filename='tsne', use_repx = False):
    #adata.var_names_make_unique()
    if use_repx:
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity, use_rep='X')
    else:    
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity)
    sc.pl.tsne(adata, color = color_group, show=False, wspace=.3)
    save_images(save_filename) 
    
def plotUMAP(adata, color_group, save_filename, use_repx = False):
    
    if use_repx:
        sc.pp.neighbors(adata, use_rep='X')
    else:    
        sc.pp.neighbors(adata,n_neighbors=10, n_pcs=20)
        
    sc.tl.umap(adata)
    sc.pl.umap(adata, color = color_group, show=False)
    save_images(save_filename)

color_group = ["cell_type","batch"]    
# Visulization of normalized data
print('scgen: ',scgen.__version__)
sc.tl.pca(corrected_adata, svd_solver='arpack')
sc.pp.neighbors(corrected_adata,n_neighbors=10, n_pcs=20)
plotUMAP(corrected_adata, color_group, 'umap_scGen_corrected_dataset1_uc3')
plotTSNE(corrected_adata, color_group, 20, 90, 'tsne_scGen_corrected_dataset1_uc3')

scgen:  1.0.0
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:00:04.55) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:00:00.24) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix
computing UMAP
    finished (0:00:02.51) --> added
    'X_umap', UMAP coordinates (adata.obsm)
computing tSNE
    using 'X_pca' with n_pcs = 20
    using the 'MulticoreTSNE' package by Ulyanov (2017)
    finished (0:00:04.00) --> added
    'X_tsne', tSNE coordinates (adata.obsm)
